# Exploration of OpenFoodFact dataset

In [ ]:
#!pip3 install --upgrade pingouin --quiet

This dataset is made up of nutritional information on food items packages, that consumers input thanks to their compter or smartphone.

Since it is an open source international data set, data may likely need cleaning, because of potential typos, non-homogenous label system, incomplete information, etc.

After these cleaning steps, we'll dive deeper in the analysis of the data to understand their structure and potential correlations.

In [ ]:
import numpy as np
import pandas as pd
from sklearn import decomposition, preprocessing
import pingouin
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import copy
import itertools
from datetime import datetime

pd.set_option('display.max_columns', None)

In [ ]:
interesting_columns = ['product_name', 'generic_name', 'categories', 'main_category', 'nutrition_grade_fr', 'ingredients_text',
                    'energy_100g','proteins_100g','carbohydrates_100g', 'sugars_100g',
                    'fat_100g', 'saturated-fat_100g', 'fiber_100g', 'sodium_100g']
labels = dict(zip(['energy_100g','proteins_100g','carbohydrates_100g', 'sugars_100g',
                    'fat_100g', 'saturated-fat_100g', 'fiber_100g', 'sodium_100g'], 
                 ['Energy', 'Proteins', 'Carbohydrates', 'Sugars', 'Fat', 'Saturated fat', 'Fibers', 'Sodium']))
quantitative_columns = [col for col in labels.keys() if '100g' in col]


In [ ]:
original_data = pd.read_csv('products.csv', sep='\t', usecols=interesting_columns)[interesting_columns]

In [ ]:
data = original_data.copy()
original_size = data.shape[0]

-----------------------------------
# 1. Data cleaning

----------------------
## 1.1 Overview

In [ ]:
print(f'The dataset contains {data.shape[0]} rows and {data.shape[1]} columns.')

<br/>Here is an excerpt of the dataset:<br/>

In [ ]:
data.head()

<br/>Let's see the main characteristics.<br/>

In [ ]:
data.describe(include='all').style.format(dict.fromkeys(quantitative_columns, '{:.2f}'))

---------
## 1.2 Cleaning columns

### 1.2.1 Column 'categories'

<br/>The '**categories**' column has much more unique values than the '**main_category**' and very similar number of missing values.  
So we can drop this column that is less relevant than "main_category".
<br/>

In [ ]:
data = data.drop(columns=['categories'])

In [ ]:
percentage_of_null_values = pd.DataFrame(data.isnull().sum()/len(data), columns = ["Missing values"])
percentage_of_null_values.sort_values(by='Missing values', ascending=False, inplace=True)
percentage_of_null_values.style.format({"Missing values": '{:.2%}'})

### 1.2.2 Column 'generic_name'

<br/>The column '**generic_name**' has more than 83% of missing values.  
We may drop it, but beforehand, use its values to fill the 'product_name' value when missing.

In [ ]:
no_product_name_but_generic_name = data[data.product_name.isna() & data.generic_name.notna()]
data.loc[no_product_name_but_generic_name.index, 'product_name'] = data.loc[no_product_name_but_generic_name.index, 'generic_name']

<br/>Check on the first 5 rows that the values were properly replaced:

In [ ]:
data.iloc[no_product_name_but_generic_name.index].head()

<br/>Now we can drop the 'generic_name' column.

In [ ]:
data = data.drop(columns=['generic_name'])

------------------------
## 1.3 Cleaning rows

### 1.3.1 Duplicate values  

In [ ]:
# This class provides a tool to plot repeatedly a pie chart that includes a new portion each time.

class ProgressivePie:
    def __init__(self, title, original_size, colors):
        self.original_size = original_size
        self.colorset = colors
        self.title = title
        self.existing_labels = []
        self.values = [self.original_size]
        self.labels = [""]
        self.colors = [self.colorset[0]]

    def _increment_data(self, new_value, new_label):
        self.values = [new_value] + self.values[:-1] + [self.values[-1] - new_value]
        self.labels = [new_label] + self.labels[:-1] + [f'{self.values[-1]} rows']
        self.colors = [self.colorset[len(self.colors)%len(self.colorset)]] + self.colors

    def _plot_pie_chart(self):
        fig1, ax1 = plt.subplots()
        ax1.pie(self.values, labels=self.labels, explode=[0.1]+[0]*(len(self.values)-1), autopct='%1.1f%%', colors=self.colors)
        ax1.set_title(self.title + f' \n(total number of lines = {self.original_size})')
        plt.show()

    def plot(self, rows, label):
        num_rows = rows.shape[0]
        if label in self.existing_labels:
            self._plot_pie_chart()
        else:
            self._increment_data(num_rows, f'{label.capitalize()} ({num_rows} rows)')
            self._plot_pie_chart()
            self.existing_labels.append(label)
   


In [ ]:
pp = ProgressivePie(original_size=data.shape[0],
                    title='Percentage of droppable values in dataset',
                    colors=['tab:green', 'tab:red', 'tab:cyan', 'tab:orange', 'tab:blue', 'tab:brown', 'tab:purple', 'tab:olive', 'tab:gray', 'tab:pink'])

<br/>How many values are duplicates ?

In [ ]:
duplicate_lines = data[data.duplicated()]
pp.plot(duplicate_lines, "duplicate lines")

In [ ]:
data = data.drop(duplicate_lines.index)
print(f'After dropping duplicate values, dataframe contains {data.shape[0]} rows.')

<br/>

### 1.3.2 Missing data  

For the purpose of analysis, we will need complete quantitative data.  
Partially filled lines may exist since a food item does not always contain all nutrients, but lines with no quantitative data at all can be dropped.  
What proportion do they represent?<br/><br/>

In [ ]:
missing_lines = data[data[quantitative_columns].isna().all(axis=1)]
pp.plot(missing_lines, "lines missing all quantitative values")

In [ ]:
data = data.drop(missing_lines.index)
print(f'After dropping rows with no quantitative data, dataframe contains {data.shape[0]} rows.')

<br/>Remaining lines are supposed to have at least one relevant quantitative value.The more we have, the better. But they are not equally important.  

a) The most important data are '**proteins_100g**', '**carbohydrates_100g**' and '**fat_100g**'. They are essential, so any NaN value in these fields result in dropping the row.

b) '**energy_100g**' can be inferred from the previous 3 values (energy = 37 \* fat + 17 \* proteins + 17 \* sugar). 

c) '**sugars_100g**' and '**saturated_fat_100g**' are subset of 'carbohydrates_100g' and 'fat_100g' respectively. And as for '**fiber_100g**', '**sodium_100g**', they are interesting information to perform a more detailed analysis but are not essential. To unify the datatype, we will transform in 0 (zero) values all NaN values for these columns.

Let's process the data successively, according to these rules.<br/><br/>

#### 1.3.2.a Dropping data with missing essential values

In [ ]:
missing_essential_lines = data[data[['proteins_100g', 'carbohydrates_100g', 'fat_100g']].isna().any(axis=1)]
pp.plot(missing_essential_lines, "lines missing essential values")

In [ ]:
data = data.drop(missing_essential_lines.index)
print(f'After dropping rows with missing essential values, dataframe contains {data.shape[0]} rows.')

<br/>

#### 1.3.2.b Replacing NaN values in "energy_100g" by (37 \* fat + 17 \* proteins + 17 \* carbohydrates)

<br/>This operation does not change the total lines number.<br/><br/>

In [ ]:
energy_nan = data[data['energy_100g'].isna()]
def calc_energy(row): 
    return row['fat_100g'] * 37 + row['proteins_100g'] * 17 + row['carbohydrates_100g'] * 17

data['energy_100g'] = data['energy_100g'].fillna(value = calc_energy(data))

#### 1.3.2.c Replacing NaN values by 0 in other quantitative columns

<br/>Again, no line is deleted in this step.<br/>

In [ ]:
data.loc[:,quantitative_columns] = data.loc[:,quantitative_columns].fillna(0)

Now there is no NaN any more in quantitative columns.

### 1.3.3 Incorrect data  

<br/>Let's look at central tendencies and dispersions for quantitative data. <br/><br/>

In [ ]:
data.describe().style.format(dict.fromkeys(quantitative_columns,'{:.2f}'))

<br/>How can we identify and handle inconsistent values?

1. Except for 'energy_100g', the values correspond to the number of **grams** of each nutrient **in 100 grams** of considered food item. **Values should then be comprised between 0 and 100**. We observe that some nutrients present negative minima, and all values have maxima much higher. These data are to be dropped. 

2. **The value of saturated fat cannot exceed that of global fat**. Same case for **sugars** with respect to carbohydrates.

3. **The sum of fat, carbohydrates and proteins cannot exceed 100 grams** [(source)](https://fr.openfoodfacts.org/questions-frequentes). Rows that do not fulfill this condition should be dropped as well.

4. Regarding energy, its **highest possible value (case of pure vegetal oil) is 3700 kJ** or 900 kcal [(source)](https://en.wikipedia.org/wiki/Food_energy). The unit is not specified in OpenFoodFact documentation, so we'll use the highest value, that is 3700 kJ, especially as the kJ is the official unit. Hence, any row with energy value beyond this limit or below 0 will be dropped.

All these tests can be grouped in a function to be applied in one go.


In [ ]:
# Creating a function to identify rows that do not fulfill all criteria above

def irrelevant(row, max_energy=3700, max_weight=100, energy_column='energy_100g',
                list_of_cols_to_sum = ['proteins_100g', 'fat_100g', 'carbohydrates_100g'],
                nutrient_columns = [col for col in quantitative_columns if col != 'energy_100g']):
    return (
        row[energy_column] > max_energy or
        (row[nutrient_columns] > 100).any() or
        (row[quantitative_columns] < 0).any() or
        np.floor(sum(row[list_of_cols_to_sum])) > max_weight or
        row['saturated-fat_100g'] > row['fat_100g'] or
        row['sugars_100g'] > row['carbohydrates_100g']
    )

# Create examples for different cases:

row_ok = pd.Series(data={'product_name': 'biscuit', "generic_name": None, "main_category": None, 
                         'nutrition_grade_fr': None, 'ingredients_text': "some ingredients",
                         'energy_100g': 1000,'proteins_100g':13.2, 'carbohydrates_100g':55.8,
                         'sugars_100g': 12, 'fat_100g':30.4, 'saturated-fat_100g': 20.5,
                         'fiber_100g':3, 'sodium_100g':4}, name="row_ok")
row_wrong_energy = copy.deepcopy(row_ok)
row_excess_value = copy.deepcopy(row_ok)
row_neg_value = copy.deepcopy(row_ok)
row_wrong_sum = copy.deepcopy(row_ok)
row_wrong_sat_fat = copy.deepcopy(row_ok)
row_wrong_sugar = copy.deepcopy(row_ok)

row_wrong_energy['energy_100g'] = 5000
row_excess_value['carbohydrates_100g'] = 120
row_neg_value['sodium_100g'] = -1
row_wrong_sum['proteins_100g'] = 95
row_wrong_sat_fat['saturated-fat_100g'] = 31
row_wrong_sugar['sugars_100g'] = 60

# Testing

names = ['row ok', 
         'row with negative value',
         'row with excessive value', 
         'row with wrong sum', 
         'row with wrong energy', 
         'row_wrong_sat_fat', 
         'row_wrong_sugar']
rows = [row_ok, 
        row_neg_value,
        row_excess_value, 
        row_wrong_sum, 
        row_wrong_energy, 
        row_wrong_sat_fat, 
        row_wrong_sugar]
results = [False, True, True, True, True, True, True]

tests_OK = not irrelevant(row_ok) and all([irrelevant(row) for row in rows[1:]])
if not tests_OK:
    print('Tests failed, please check')
    for name, row, result in zip(names, rows, results):
        if result != irrelevant(row):
            print(f'Testing {name}, expect function to return {result}, result: ',irrelevant(row))

Let's clean the dataset (the process takes up to 7-9 min on Kaggle).

In [ ]:
start = datetime.now()
print("Cleaning started at", start.strftime('%H:%M:%S'))
data['irrelevant'] = data.apply(irrelevant, axis=1)
end = datetime.now()
print('Ended at', end.strftime('%H:%M:%S'))
print('Elapsed time:', end - start)

In [ ]:
irrelevant_lines = data[data['irrelevant']]
pp.plot(irrelevant_lines, 'irrelevant lines')

In [ ]:
data = data.drop(irrelevant_lines.index)
data = data.drop('irrelevant', axis=1)
print(f'After dropping rows where values are irrelevant, dataframe contains {data.shape[0]} rows.')

In [ ]:
dropped = (original_size - data.shape[0])/original_size
print(f"After this cleaning session, {'{:.2%}'.format(dropped)} of initial data have been dropped for being duplicate, incomplete or irrelevant.") 

<br/>To improve readability in graphs, we'll also replace by '' all NaN fields in qualitative columns and shorten ingredient list to 30 characters.<br/><br/>

In [ ]:
qualitative_columns = [col for col in data.columns if col not in quantitative_columns]
data.fillna(dict.fromkeys(qualitative_columns, ''), inplace=True)
data['ingredients_text']= data['ingredients_text'].str.slice(0,30)
data.head()

<br/>Now we can visualize statistics on our cleaned dataset.<br/><br/>

In [ ]:
data.describe().style.format(dict.fromkeys(quantitative_columns, '{:.2f}'))

<br/>To visualize these figures, we'll use boxplots, that show on the same graph the median, quartiles and outliers (i.e. values that are distant to the median for more than 1,5 * the interquartile distance).<br/><br/>

In [ ]:
# Define a 2*4 grid for subplots

rows, cols = 2, 4
fig = make_subplots(rows=rows, cols=cols)

# Plot boxplots for all quantitative columns

grad = list(itertools.product(range(1,rows+1), range(1, cols+1)))
for (row, col), variable in zip(grad, quantitative_columns):
    fig.add_trace(
        go.Box(y=data[variable], name=labels[variable], boxmean=True),
        row=row, col=col)
fig.update_layout(height=800, width=1000,
                  title_text="Nutrients quantity per 100g", showlegend=False)    
fig.show()   

<br/>The boxplots are "squashed" because of many zero or outliers values, especially for sodium. Let's look at those values more precisely, by displaying only products where the values are > 0.<br/><br/>

In [ ]:
for col, color in zip(quantitative_columns, px.colors.qualitative.Plotly):
    partial_data = data[data[col]>0][['product_name', 'ingredients_text', col]]

    fig = px.histogram(partial_data, x=col, nbins=100, color_discrete_sequence=[color],
                       marginal="box", hover_data=['product_name', 'ingredients_text'])
    fig.show() 

-------------------------------
# 2. Multivariate Analysis

--------------------------
## 2.1. Matrix of correlation

In [ ]:
correlations = data.corr()

In [ ]:
px.imshow(correlations, color_continuous_scale = 'Blues')

<br/>We can see that energy is highly correlated to fat, carbohydrates and proteins, and that:  
- carbohydrates and sugars, on one hand,  
- fat and saturated fat, on the other hand,  
are closely related.
We can also notice that sodium is in no way related to other nutrients.<br/><br/>

We must keep in mind that the main nutrients (fat, carbohydrates and proteins) are correlated, since their sum is limited to 100. This can be seen if we plot food items in a 3D space based on these 3 nutrients vectors. We get a tetrahedron (plot made from a sample od 50 000 food items):

In [ ]:
sample = data.sample(50000)

In [ ]:
px.scatter_3d(sample, x='fat_100g', y='carbohydrates_100g', z='proteins_100g', color="nutrition_grade_fr",
             category_orders = {'nutrition_grade_fr': ['a', 'b', 'c', 'd', 'e']}
             )

--------------------
## 2.2. PCA

<br/>We'll use a Principal Components Analysis to determine which factors, among the quantitative values of the dataset, describe it best.  
We'll exclude the energy value, since it is dependant of other values (made up of fat, carbohydrates and proteins).

In [ ]:
# Define parameters

nutrient_columns = [col for col in quantitative_columns if col != 'energy_100g']
n_comp = len(nutrient_columns)
data_pca = data[nutrient_columns]
X = data_pca.values
names = data_pca.index
features = data_pca.columns

# Normalize values

scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# Apply PCA

pca = decomposition.PCA(n_components=n_comp)
pca_data = pca.fit(X_scaled)
pc_index = [f'PC{i+1}' for i in range(n_comp)]

In [ ]:
cum_var = pd.DataFrame(
    np.cumsum(pca_data.explained_variance_ratio_), 
    columns = ['Cumulative explained variance'], 
    index = pc_index
)

In [ ]:
x = cum_var.index
x.name = "Components"
fig = px.line(cum_var, 
              x=x, 
              y="Cumulative explained variance", 
              title='Cumulative explained variance',
              hover_data={'Cumulative explained variance': ':.2%'}
             )
fig.update_traces(mode="markers+lines")
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

<br/>We observe that the first 2 components explain only just more than 50%, and 4 components explain more 80% of the variance. More specifically:<br/><br/>

In [ ]:
principal_components = pd.DataFrame(data=pca_data.components_, columns=nutrient_columns, index = pc_index)
complete = principal_components.copy()
complete['Explained variance'] = pca_data.explained_variance_ratio_
complete['Cumulative explained variance'] = cum_var['Cumulative explained variance']

complete

In [ ]:
loadings = principal_components.T* np.sqrt(pca_data.explained_variance_)

fig= fig = go.Figure(
    layout_title_text="Nutrients represented in the first vectorial plan (P1, P2)",
)
fig.update_xaxes(range=[-1, 1])
fig.update_yaxes(range=[-1, 1])
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)

for (i, nutrient), color in zip(enumerate(nutrient_columns), px.colors.qualitative.Plotly):
    fig.add_shape(
        type='line',
        x0=0, y0=0,
        x1=loadings.iloc[i, 0],
        y1=loadings.iloc[i, 1],
        line=dict(color=color,width=2)
    )
    fig.add_annotation(
        x=loadings.iloc[i, 0],
        y=loadings.iloc[i, 1],
        ax=0, ay=0,
        xanchor="left",
        yanchor="bottom",
        text=nutrient,
    )
    
fig.show()    

As we could expect, we notice that sugars and carbohydrates are very close, as well as fat and saturated fat.

---------------------
## 2.3 ANOVA

The nutriscore was created to reflect the overall "quality" of food items. Beyond the only caloric intake, it is aimed to take into account the levels of sodium, fibers, saturated fats, sugar, etc.  

Here we are going to analyze how the nutriscore, which is a qualitative value scoring from a to e, has an influence on quantitative values. By doing this we may be able to make hypothesis on the definition of the nutriscore. 

In [ ]:
nutriscore = data[data['nutrition_grade_fr']!=""]
nutriscore.groupby("nutrition_grade_fr").mean().style.format('{:.2f}')

<br/>We can see there is a "correlation" between nutrition grade and some nutrients, but since nutrition grade is a qualitative value, this correlation has to be studied through an Analysis of Variance (ANOVA.)<br/><br/>

In [ ]:
for item in quantitative_columns:
    fig = px.box(nutriscore,x='nutrition_grade_fr', y=item, 
                 title=f'Distribution of {item} for each nutriscore value',
                 labels={'nutrition_grade_fr': "Nutriscore"}, 
                 color = 'nutrition_grade_fr',
                 category_orders={'nutrition_grade_fr': ["a", "b", "c", "d", "e"]},
                )
    fig.update_layout(showlegend=False)
    fig.show()

<br/>Since the fibers and sodium values are the one less visible in the boxplot, because of the outliers, we hope that performing ANOVA will allow us to check whether they depend on nutriscore.<br/><br/>

In [ ]:
# Normalize nutriscore dataframe

to_normalize = nutriscore[quantitative_columns].values
scaler = preprocessing.StandardScaler().fit(to_normalize)
normalized = scaler.transform(to_normalize)
nutriscore_norm = nutriscore.copy()
nutriscore_norm[quantitative_columns] = normalized

# Create a dataframe to stack results

columns = ['item', 'F', 'np2']
results = pd.DataFrame(columns=columns)

for item in quantitative_columns:
    anova =  pingouin.anova(data=nutriscore_norm, dv=item, between='nutrition_grade_fr')
    row = pd.DataFrame(np.array([[item, anova.loc[0,'F'],anova.loc[0, 'np2']]]), columns=columns)
    results = pd.concat([results, row])

# Format and sort dataframe

results.loc[:,['F', 'np2']] = results.loc[:,['F', 'np2']].astype(float)
results.sort_values(by="F", ascending=False, inplace=True)
results.reset_index(inplace=True)
results.drop('index', axis=1, inplace = True)
results.style.format({'F': '{:.2f}', 'np2':'{:.2f}'})

- The F score represents the variance interclass divided by the variance intraclass. A hig score shows that there are much more variations between classes than in each class, that is to say, the different classes do have an impact. 
- The eta-square (np2) score represents the variance interclass divided by the overall variance. 

<br/>The chart above shows that:  
- the nutrition score is a relevant parameter with respect to saturated fat, energy, fat and sugar  
- on the other hand, the content in carbohydrates, fibers, proteins and sodium is little or very little influenced by the nutriscore.

The differences on fat, saturated fat and sugars come as no suprise given the concept of Nutriscore. Neither the small score of proteins that is neither a "bad" or "good" nutrient.  
But the small differences on fibers and sodium are surprising. We could have thought that they would be more strongly related to the nutriscore.

We may conclude that the relative importances of fat and sugar in nutriscore are higher than that of sodium.

Another explanation could be that while it is **sodium** that is Monitored by nutriscore, some labels mention the **salt**. Howerver, for 100g of salt, there is only 40g of sodium [(source)](https://www.nephrohug.ch/2016/04/20/le-sel-le-sodium-quelle-difference/). And not all food labels specifiy "sodium" : many of them mention "salt". If consumers mention the salt rate instead of the sodium rate in the database, then it can skew the data.


We may also be aware of the fact that nutriscores are no represented equally in the database. The highest scores c, d and e are overrepresented:

In [ ]:
px.histogram(data, x="nutrition_grade_fr",
             color = 'nutrition_grade_fr',
             category_orders={'nutrition_grade_fr': ['', 'a', 'b', 'c', 'd', 'e']},
             labels= {'':"no score", "a": "A", "b": "B", "c": "C", "d": "D", "e":"E"},  
             title='distribution of nutrition grade',
            )